**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning 

# Pirashanth RATNAMOGAN - MVA 2018

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

This function returns what action the agent will perform knowing the current state. This choice is based on all the previous cumulated knowledge. In order to not make the actions being too biased by the past **during the training phase**, it is usual to add an exploration criteria (not act as suggested by what we have learned but act randomly). The epsilon parameter allows to control the exploration. This is the $\epsilon$-greedy strategy. If epsilon = 0, then we will follow the action based on what we have learned only. If epsion =1, then we will act randomly at each time step. This parameter is essential and must be setted carefully.

The epsilon parameter is essential: indeed we can imagine a situation with two actions,two states and a discount equals to one. Playing the first action gives a reward of 1/4 with probability one, playing the second action gives a reward of 1 with probability 1/2. With probability 1/2 we will only use the first action (if when we have played the second action we have obtained a zero) while playing always the second action would give a mean reward of 1/2 while playing always the first action would give a mean reward of 1/4. Hence it would have been a better choice to always play the second action. In order to not being biased by such situations where the past is disadvantageous for one action the epsilon parameter allows to keep letting a chance to all the actions to improve how they are perceived from our algorithm.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0 #Correct typo here
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #Correct typo here

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array contains an array representing where the rat can go. It has -1 in the boundaries where we can't go, 0 where we can actually go and finally 1 in the position where the rat is.
The board represent the reward associated to each cell of the grid. If our rat comes to the position $(x,y)$ it will get the reward board[x,y].

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
 
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        
        while not game_over:
            
            action = agent.act(state,train=False) #this is the main point, during the test phase we don't have to use the epsilon greedy approach
            
            prev_state= state
            state,reward,game_over = env.act(action)
            if reward >0:
                win = win +reward 
            if reward < 0:
                lose = lose-reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random5.mp4'))

Win/lose count 12.0/13.0. Average score (-1.0)
Win/lose count 11.0/13.0. Average score (-1.5)
Win/lose count 13.5/21.0. Average score (-3.5)
Win/lose count 12.0/12.0. Average score (-2.625)
Win/lose count 10.0/21.0. Average score (-4.3)
Win/lose count 8.5/7.0. Average score (-3.3333333333333335)
Win/lose count 12.0/15.0. Average score (-3.2857142857142856)
Win/lose count 15.0/16.0. Average score (-3.0)
Win/lose count 8.5/9.0. Average score (-2.7222222222222223)
Win/lose count 7.5/20.0. Average score (-3.7)
Win/lose count 12.0/18.0. Average score (-3.909090909090909)
Final score: -3.90909090909


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




# Answer

As seen in class we have 

\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
& =  \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

Then we have:

\begin{align}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= \max_{\pi}r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',a')\\
&= r(s,a) + \gamma \max_{\pi} \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \qquad \text{with } \pi=(a',\pi')
\end{align}

In order to exchange the maximum and the sum one has to remark the following:
We have first the trivial inequality

\begin{equation*}
\max_{\pi} \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') \leq \sum_{(s',a')} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi'}(s',a') 
\end{equation*}

Then we have considering $\hat{\pi}(s',a') = \arg\max_{\pi'} Q^{\pi'}(s',a')$
\begin{align*}
 \sum_{(s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi'}Q^{\pi'}(s',a') &= \sum_{(s')} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\hat{\pi}(s',a')}(s',a') \\
 &\leq \max_{\pi'} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a') 
\end{align*}

So we can rewrite our initial equation as :
\begin{align}
Q^{*}(s,a)&= r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

The proposed objective function 

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

Allows to enforce the previous equality, if our neural network perfectly converge this equality must be true. Using this loss function we try to enforce our learned Q-value to verify the Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [15]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]
            

    def random_access(self):
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [16]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            # We will only try to improve the prediction where we actually have a new information
            #that means that when we have no information we will keep the previous prediction of the  network
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) #simple one hidden layer network (see performance of other methods below)
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [110]:
epochs_train = 50 #I have used epochs train and epochs test equals to 50 in the two questions below
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0069 | Win/lose count 2.0/3.0 (-1.0)
Epoch 001/050 | Loss 0.0283 | Win/lose count 8.5/9.0 (-0.5)
Epoch 002/050 | Loss 0.0091 | Win/lose count 7.5/4.0 (3.5)
Epoch 003/050 | Loss 0.0138 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/050 | Loss 0.0114 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/050 | Loss 0.0117 | Win/lose count 0.5/0 (0.5)
Epoch 006/050 | Loss 0.0527 | Win/lose count 9.5/10.0 (-0.5)
Epoch 007/050 | Loss 0.0087 | Win/lose count 1.5/6.0 (-4.5)
Epoch 008/050 | Loss 0.0092 | Win/lose count 0.5/2.0 (-1.5)
Epoch 009/050 | Loss 0.0044 | Win/lose count 2.0/0 (2.0)
Epoch 010/050 | Loss 0.0617 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/050 | Loss 0.0037 | Win/lose count 7.0/6.0 (1.0)
Epoch 012/050 | Loss 0.0097 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/050 | Loss 0.0149 | Win/lose count 7.0/1.0 (6.0)
Epoch 014/050 | Loss 0.0623 | Win/lose count 3.0/9.0 (-6.0)
Epoch 015/050 | Loss 0.0105 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/050 | Loss 0.0043 | Win/lose count 3.0/1.

# Comparison of various deep neural network architectures (fully connected) and various parameters

I tried 6 different architectures (the size of the neurons doesn't seems to play an important role so I have decided to keep them to a constant size of 30):
* Architecture 1 - 1 hidden layers with output of size 30
* Architecture 2 - 1 hidden layers with output of size 30 and batchnormalization in between them
* Architecture 3 - 2 hidden layers with output of size 30
* Architecture 4 - 2 hidden layers with output of size 30 and batchnormalization in between them
* Architecture 5 - 3 hidden layers with output of size 30 
* Architecture 6 - 3 hidden layers with output of size 30 and batchnormalization in between them

There is some stochasticity in the problem however the printed score represents the mean outcome over 50 tests.
I tried to play with the learning rate and the epsilon parameter that seems to both play an important role. I could have also tried various optimizers, batch size and memory size but those parameters seems to have a lower importance and the following table would have been too long. I could have also tried various regularization technics such as Dropout and try various clipping parameters...




| Architecture       |     Epochs Train    |        Epsilon | Learning Rate | Batch size|Memory size| Final Test Score (Average over 50 epochs)
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
| 1       |     50      |         0.5 |0.1 |32 |2000 |2.55 |
| 2   |   50      |         0.5 |0.1 |32 |2000 |1.45 |
| 3       |     50      |         0.5 |0.1 |32 |2000 |2.21 |
| 4       |      50      |         0.5 |0.1 |32 |2000 |1.35  |
| 5     |      50      |         0.5 |0.1 |32 |2000 |2.625 |
| 6     |     50      |         0.5 |0.1 |32 |2000 |0.39|
| 1       |     50      |         0.5 |0.5 |32 |2000  | 2.21 |
| 2   |   50      |         0.5 |0.5 |32 |2000  | 1.6 |
| 3       |       50      |         0.5 |0.5 |32 |2000  |-0.62 |
| 4       |      50      |         0.5 |0.5 |32 |2000  | -0.76 |
| 5     |     50      |         0.5 |0.5 |32 |2000  | 1.16 |
| 6     |     50      |         0.5 |0.5 |32 |2000  | -0.28 |
|** 1  **     |     ** 50   **   |       **  0.1 **|**0.1** |**32** |**2000**  |**3.01** |
| 2   |  50      |         0.1 |0.1 |32 |2000  | 2.71 |
| 3       |      50      |         0.1 |0.1 |32 |2000  |1.6 |
| 4       |      50      |         0.1 |0.1 |32 |2000  |0.28 |
| 5     |     50      |         0.1 |0.1 |32 |2000  |2.32 |
| 6     |      50      |         0.1 |0.1 |32 |2000  |-0.47|

As one can see, in our test we have obtained the best outcome using a simple network with only one hidden layer. From our test it seems to be a bad idea to set the learning rate too high and to use some BatchNormalization layers that are not adapted for the Deep Reinforcement learning task.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [12]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [112]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0026 | Win/lose count 6.5/3.0 (3.5)
Epoch 001/050 | Loss 0.0013 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/050 | Loss 0.0083 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/050 | Loss 0.0068 | Win/lose count 4.5/3.0 (1.5)
Epoch 004/050 | Loss 0.0028 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/050 | Loss 0.0023 | Win/lose count 3.5/3.0 (0.5)
Epoch 006/050 | Loss 0.0019 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/050 | Loss 0.0015 | Win/lose count 6.5/0 (6.5)
Epoch 008/050 | Loss 0.0017 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/050 | Loss 0.0022 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/050 | Loss 0.0055 | Win/lose count 5.5/2.0 (3.5)
Epoch 011/050 | Loss 0.0070 | Win/lose count 7.5/5.0 (2.5)
Epoch 012/050 | Loss 0.0040 | Win/lose count 7.5/4.0 (3.5)
Epoch 013/050 | Loss 0.0033 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/050 | Loss 0.0011 | Win/lose count 7.5/2.0 (5.5)
Epoch 015/050 | Loss 0.0013 | Win/lose count 8.5/2.0 (6.5)
Epoch 016/050 | Loss 0.0024 | Win/lose count 5.5/4.0 (1.5

# Comparison of various deep neural network architectures (with some Convolutional layers)

Here I have decided to set the learning rate and the epsilon parameters to 0.1 because it appears to be some good parameters in order to obtain a good solution to our problem. I will set the number of epoch to 50 and I will try various architecture:
* Architecture 1: 1 hidden Convolution Layer ((50,(2,2)) , relu activations
* Architecture 2: 2 hidden Convolution Layer ((50,(2,2) and then (30,(2,2)), relu activations
* Architecture 3: 2 hidden Convolution Layer ((50,(2,2),then a MaxPooling layer (2,2) then a Convolution layer (30,(2,2)), relu activations
* Architecture 4: 2 hidden Convolution Layer ((50,(2,2),and then (30,(2,2)), relu activations with Dropout (p=0.3)
* Architecture 5: 1 hidden Convolution Layer ((50,(3,3)) , relu activations

I remind that there is some stochasticity in the given test score, however from what I observed it represents approximatively the difference in performance between the models.


| Architecture       |     Epochs Train    |        Epsilon | Learning Rate | Batch size|Memory size| Final Test Score (Average over 50 epochs)|
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
| 1       |      50     |         0.1 |0.1 |32 |2000  | 6.61|
|** 2 **  | ** 50**      |         **0.1** |**0.1** |**32** |**2000**  | **8.54** |
| 3       |      50      |         0.1 |0.1 |32 |2000  | 6.21 |
| 4       |      50      |         0.1 |0.1 |32 |2000  |4.58 |
| 5     |     50      |         0.1 |0.1 |32 |2000  |6.12 |


As we can observe the suggested network performs the best with a good score of 8.54.

In the following questions we will use the two networks that have provided the best outcomes in question 7 and question 8. That means that we will use for the FC - 1 hidden layers with output of size 30 - and for the CNN - 2 hidden Convolution Layer ((50,(2,2) and then (30,(2,2)) , relu activations- 
 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [121]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 33.5/0. Average score (33.5)
Win/lose count 52.0/0. Average score (42.75)
Win/lose count 62.0/0. Average score (49.166666666666664)
Win/lose count 32.5/0. Average score (45.0)
Win/lose count 57.0/0. Average score (47.4)
Win/lose count 58.0/0. Average score (49.166666666666664)
Win/lose count 56.5/0. Average score (50.214285714285715)
Win/lose count 64.0/1.0. Average score (51.8125)
Win/lose count 66.0/1.0. Average score (53.27777777777778)
Win/lose count 56.5/0. Average score (53.6)
Win/lose count 47.5/0. Average score (53.04545454545455)
Final score: 53.0454545455
Test of the FC
Win/lose count 9.5/0. Average score (9.5)
Win/lose count 4.5/0. Average score (7.0)
Win/lose count 11.0/0. Average score (8.333333333333334)
Win/lose count 20.0/0. Average score (11.25)
Win/lose count 16.5/0. Average score (12.3)
Win/lose count 11.0/0. Average score (12.083333333333334)
Win/lose count 44.5/0. Average score (16.714285714285715)
Win/lose count 11.5/0. Average score

In [122]:
HTML(display_videos('cnn_test10.mp4'))

In [123]:
HTML(display_videos('fc_test10.mp4'))

# Comparison of both algorithms

Here I have performed various test changing the temperature parameter. I have called Best FC, the fully connected network that has provided the best outcome in the previous questions. I have called Best CNN, the network with CNN layers that has provided the best outcome in the previous questions. I have trained the networks with a temperature equal to 0.3 during 50 epochs.

 |    Temperature    |   Final Test Score Best FC (Average over 12 epochs)|Final Test Score Best CNN (Average over 12 epochs)|
| :------------ | :-------------: | -------------: | 
| ** 0.1   **     | 1.13636363636    |     2.59090909091   |
| ** 0.2   **      |   3.68181818182      |  4.72727272727 |
| ** 0.3   **     |  3.81818181818   |   8.18181818182  |
| ** 0.4   **        | 2.77272727273  |10.6818181818      |
| ** 0.5   **      |    7.5  |  16.1363636364 |
| ** 0.6   **      |   7.31818181818    |  15.6818181818 |
| ** 0.7   **      |  10.9090909091  |  29.3636363636   |
| ** 0.8   **     |   9.95454545455      |    38.7272727273 |
| ** 0.9   **     |   14.0   |   53.0454545455 |

As we can see, the performance using the architecture based on CNN is widely better than the one obtained using some fully connected networks (about 5 points more). THis gap is increasing with the temperature. 

The main issue that we can observe is that the algorithms tends to not explore the map that means that sometimes it goes to a position A then goes to a position B and then go back to position A ... This method will not decrease the score but will not help to improve it. This can be especially observed when from the position of the rat it can't observe any positive reward (hence it appears especially when the temperature is low).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [17]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    #New training procedure that tries to improve the exploration of the algorithm
    #decay_parameter_epsilon in order to use the decreasing $\epsilon$-greedy exploration
    
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [30]:
epochs_train = 50
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon =0.6, memory_size=2000, batch_size = 32,n_state=3) #epsilon is high because it will decrease with time
train_explore(agent, env, epochs_train,decay_parameter_epsilon=0.1, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0229 | Win/lose count 11.0/25.700000000000074 (-14.700000000000074)
Epoch 001/050 | Loss 0.0122 | Win/lose count 11.5/20.299999999999997 (-8.799999999999997)
Epoch 002/050 | Loss 0.0582 | Win/lose count 14.5/20.90000000000005 (-6.400000000000048)
Epoch 003/050 | Loss 0.0036 | Win/lose count 13.5/21.1 (-7.600000000000001)
Epoch 004/050 | Loss 0.0062 | Win/lose count 18.5/18.49999999999999 (1.0658141036401503e-14)
Epoch 005/050 | Loss 0.0086 | Win/lose count 10.0/25.500000000000085 (-15.500000000000085)
Epoch 006/050 | Loss 0.0534 | Win/lose count 15.0/25.000000000000007 (-10.000000000000007)
Epoch 007/050 | Loss 0.0103 | Win/lose count 14.5/23.0 (-8.5)
Epoch 008/050 | Loss 0.0052 | Win/lose count 25.5/12.899999999999975 (12.600000000000025)
Epoch 009/050 | Loss 0.0047 | Win/lose count 17.0/13.899999999999972 (3.100000000000028)
Epoch 010/050 | Loss 0.0111 | Win/lose count 16.5/22.200000000000017 (-5.700000000000017)
Epoch 011/050 | Loss 0.0048 | Win/lose count 24.

In [28]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 25.0/2.0. Average score (23.0)
Win/lose count 22.5/2.0. Average score (21.75)
Win/lose count 25.0/2.0. Average score (22.166666666666668)
Win/lose count 24.5/3.0. Average score (22.0)
Win/lose count 30.0/4.0. Average score (22.8)
Win/lose count 25.0/3.0. Average score (22.666666666666668)
Win/lose count 21.5/2.0. Average score (22.214285714285715)
Win/lose count 21.0/3.0. Average score (21.6875)
Win/lose count 21.0/1.0. Average score (21.5)
Win/lose count 25.5/1.0. Average score (21.8)
Win/lose count 22.0/2.0. Average score (21.636363636363637)
Final score: 21.6363636364


** Conclusion on the use of the approach with exploration:** As we can see the outcome has been widely improved using the approach with exploration. We have now a score that can reach an average score of 19 (over 11 epochs) with a temperature of 0.3 while without exploration the DQN_CNN trained during 50 epochs was reaching score that rarely exceed 9  (over 11 epochs during the test phase). This is a wide improvement!

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [31]:
class DQN_Mim(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mim, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #now we will not use the fact that our model need to verify the bellman equation but directly copy what the expert has done
        #hence we juste need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 10  #Now we want to mimick the expert hence we want our target function to be maximized 
            #in the action applied by the expert
            
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mim(DQN_Mim):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mim, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) #model with 1 hidden CNN performs the best
        #with my method from what I ve experimented
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [32]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [33]:
epochs_train = 50
expert = agent
agent_n = DQN_CNN_Mim(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 000/050 | Loss 3.4851 | Win/lose count 20.0/2.0 (18.0)
Epoch 001/050 | Loss 4.1354 | Win/lose count 21.5/1.0 (20.5)
Epoch 002/050 | Loss 5.5993 | Win/lose count 28.5/3.0 (25.5)
Epoch 003/050 | Loss 5.4936 | Win/lose count 14.5/2.0 (12.5)
Epoch 004/050 | Loss 6.6975 | Win/lose count 30.0/1.0 (29.0)
Epoch 005/050 | Loss 8.7608 | Win/lose count 22.0/3.0 (19.0)
Epoch 006/050 | Loss 9.7985 | Win/lose count 22.5/2.0 (20.5)
Epoch 007/050 | Loss 7.3762 | Win/lose count 22.5/4.0 (18.5)
Epoch 008/050 | Loss 7.7667 | Win/lose count 18.5/2.0 (16.5)
Epoch 009/050 | Loss 8.8907 | Win/lose count 19.0/1.0 (18.0)
Epoch 010/050 | Loss 6.0965 | Win/lose count 26.5/4.0 (22.5)
Epoch 011/050 | Loss 9.5132 | Win/lose count 23.5/0 (23.5)
Epoch 012/050 | Loss 8.2908 | Win/lose count 22.5/0 (22.5)
Epoch 013/050 | Loss 7.2360 | Win/lose count 23.5/1.0 (22.5)
Epoch 014/050 | Loss 7.5999 | Win/lose count 25.0/3.0 (22.0)
Epoch 015/050 | Loss 7.1582 | Win/lose count 15.5/3.0 (12.5)
Epoch 016/050 | Loss 8.5790 

In [34]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Win/lose count 14.0/0. Average score (14.0)
Win/lose count 8.5/2.0. Average score (10.25)
Win/lose count 18.5/2.0. Average score (12.333333333333334)
Win/lose count 6.5/1.0. Average score (10.625)
Win/lose count 16.5/2.0. Average score (11.4)
Win/lose count 15.5/2.0. Average score (11.75)
Win/lose count 4.5/0. Average score (10.714285714285714)
Win/lose count 15.0/0. Average score (11.25)
Win/lose count 5.5/0. Average score (10.61111111111111)
Win/lose count 12.5/0. Average score (10.8)
Win/lose count 11.5/2.0. Average score (10.681818181818182)
Final score: 10.6818181818


In [38]:
HTML(display_videos('cnn_test_mimick10.mp4'))

As we can see our algorithm that tries to mimick the expert performs better than the algorithm without exploration! 

The final score for the copy is about 11 (average over 11 epochs for testing). It is lower than the score obtained using the approach with exploration. This can be explained by the fact that it doesn't really learn to improve the score but it just tries to copy the original algorithm hence it could at best performs such as the expert algorithm but it couldn't exceed it. We could try an approach where we would have pretrained the algorithm without the expert and then train the algorithm to copy the expert (or the reverse procedure). 